In [6]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from glob import glob

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [7]:
trait_list = pd.read_excel("../r2-diff/results/trait_info.xlsx")["id"].values

trait_list = [
    "log_hip_circ",
    "log_leukocyte",
    "height",
    "systolic_BP",
    "diastolic_BP",
    "LDL",
    "waist_hip_ratio",
    "log_waist_circ",
    "log_HDL",
    "log_BMI",
    "log_triglycerides",
    "years_of_edu",
]

In [8]:
DATA_DIR = "/u/scratch/y/yiding/prs/ukbb-aou/out/"

In [9]:
trait_list = [p.split("/")[-1] for p in glob(DATA_DIR + "*")]

In [10]:
trait_list

['systolic_BP',
 'log_leukocyte',
 'log_triglycerides',
 'log_HDL',
 'LDL',
 'diastolic_BP',
 'height',
 'log_CRP',
 'neuroticism',
 'cholesterol',
 'log_BMI',
 'years_of_edu',
 'waist_hip_ratio']

In [11]:
df_pgs = None
dict_columns = {
    "CHR": "CHROM",
    "A1": "ALT",
    "A2": "REF",
    "SNP": "SNP",
    "POS": "POS",
    "WEIGHT": "WEIGHT",
}

for trait in tqdm(trait_list):
    weights_path = os.path.join(
        DATA_DIR, f"{trait}/{trait}.auto.weight.aou_snp_id.tsv.gz"
    )
    df_weights = pd.read_csv(weights_path, sep="\t")
    sample_cols = [col for col in df_weights.columns if col.startswith("SAMPLE_")]
    df_weights["WEIGHT"] = df_weights[sample_cols].mean(axis=1)
    df_weights = (
        df_weights[dict_columns.keys()]
        .rename(columns=dict_columns)
        .rename(columns={"WEIGHT": trait})
    )
    if df_pgs is None:
        df_pgs = df_weights
    else:
        assert df_weights.iloc[:, 0:5].equals(df_pgs.iloc[:, 0:5])
        df_pgs[trait] = df_weights[trait]

100%|██████████| 13/13 [12:48<00:00, 59.09s/it]


In [13]:
df_pgs.to_csv("results/pgs_weights.v1.tsv.gz", sep="\t", index=False)